In [3]:
#load training data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from data_preprocess.adult import load_adult



train_data, test_data, data_info,cat_vars = load_adult()


Data info: [(1, 'tanh', 'Age'), (7, 'softmax', 'WorkClass'), (16, 'softmax', 'EducationNum'), (2, 'softmax', 'MaritalStatus'), (14, 'softmax', 'Occupation'), (6, 'softmax', 'Relationship'), (5, 'softmax', 'Race'), (2, 'softmax', 'Gender'), (3, 'softmax', 'CapitalGain'), (3, 'softmax', 'CapitalLoss'), (45, 'softmax', 'HoursPerWeek'), (2, 'softmax', 'NativeCountry'), (2, 'softmax', 'Income')]


In [4]:
train_data.head()

,Age,WorkClass,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
33716,27,Local-gov,10,Couple,Adm-clerical,Own-child,White,Female,zero,zero,17.0,US,<=50K
16224,50,Private,9,Single,Craft-repair,Unmarried,White,Female,zero,zero,17.0,US,<=50K
24357,50,Private,9,Single,Craft-repair,Unmarried,White,Male,zero,low,22.0,US,<=50K
44698,42,Private,9,Single,Exec-managerial,Not-in-family,White,Female,zero,zero,17.0,US,<=50K
37191,20,Private,6,Single,Craft-repair,Own-child,White,Male,zero,zero,17.0,US,<=50K


In [6]:
#train GAN
# support the follwoing net:
#    'vanilla_VAE',
#    'WGANGP',
#    'dp_WGanSynthesizer', differentially private wgan with weight clipping
#    'CTGANSynthesizer',
#    'TVAESynthesizer',
#    'constrained_WGANGP', our proposed defense

from Synthesizers import *
from Synthesizers.train import train_Gmodel,sample_data_fromGen
gan,tf = train_Gmodel(WGANGP,train_data,data_info, seed=0,epochs=300,bs=500)

data dim: 108
Begin training...
[0/300]	  Loss_D: -0.1669	Loss_G: -0.7255	
[1/300]	  Loss_D: -0.0799	Loss_G: -0.4336	
[2/300]	  Loss_D: -0.0147	Loss_G: -0.1678	
[3/300]	  Loss_D: -0.0638	Loss_G: -0.3210	
[4/300]	  Loss_D: -0.0522	Loss_G: -0.2776	
[5/300]	  Loss_D: -0.0274	Loss_G: -0.3340	
[6/300]	  Loss_D: -0.0643	Loss_G: -0.3530	
[7/300]	  Loss_D: -0.0303	Loss_G: -0.3409	
[8/300]	  Loss_D: -0.0155	Loss_G: -0.2879	
[9/300]	  Loss_D: -0.0454	Loss_G: -0.2719	
[10/300]	  Loss_D: -0.0307	Loss_G: -0.3138	
[11/300]	  Loss_D: -0.0413	Loss_G: -0.2829	
[12/300]	  Loss_D: -0.0607	Loss_G: -0.2753	
[13/300]	  Loss_D: -0.0129	Loss_G: -0.2331	
[14/300]	  Loss_D: -0.0434	Loss_G: -0.1851	
[15/300]	  Loss_D: -0.0943	Loss_G: -0.1718	
[16/300]	  Loss_D: -0.0443	Loss_G: -0.2165	
[17/300]	  Loss_D: -0.0602	Loss_G: -0.1766	
[18/300]	  Loss_D: -0.0619	Loss_G: -0.2195	
[19/300]	  Loss_D: -0.0380	Loss_G: -0.2346	
[20/300]	  Loss_D: -0.0809	Loss_G: -0.3056	
[21/300]	  Loss_D: -0.0911	Loss_G: -0.3436	
[22/300]	 

In [7]:
syn = sample_data_fromGen(gan.myG.cpu(),tf,n=int(len(train_data)),data_info=data_info,benchmark='Adult',seed=0)

Begin sample，seed= 0
Inverse transform completed!


In [8]:
syn.head()

,Age,WorkClass,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
0,35.0,Private,7.0,Single,Sales,Own-child,White,Male,zero,zero,20.0,US,<=50K
1,29.0,Self-emp-not-inc,9.0,Couple,Farming-fishing,Husband,White,Male,zero,zero,31.0,US,>50K
2,45.0,Private,6.0,Couple,Craft-repair,Husband,White,Male,zero,zero,17.0,US,<=50K
3,40.0,Local-gov,10.0,Couple,Other-service,Husband,White,Male,zero,zero,17.0,US,<=50K
4,45.0,Private,4.0,Couple,Machine-op-inspct,Wife,White,Female,zero,zero,17.0,US,>50K


In [9]:
from attacks.TableGAN_MCA import Tablegan_mca
labeled_syn = Tablegan_mca(train_data,
                            syn,
                            data_info,
                            WGANGP,
                            epochs=300, 
                            benchmark='Adult',
                            seed=0,
                            n_bins=60)

Synthetic datasets info: [(1, 'tanh', 'Age'), (6, 'softmax', 'WorkClass'), (16, 'softmax', 'EducationNum'), (2, 'softmax', 'MaritalStatus'), (13, 'softmax', 'Occupation'), (6, 'softmax', 'Relationship'), (5, 'softmax', 'Race'), (2, 'softmax', 'Gender'), (3, 'softmax', 'CapitalGain'), (3, 'softmax', 'CapitalLoss'), (44, 'softmax', 'HoursPerWeek'), (2, 'softmax', 'NativeCountry'), (2, 'softmax', 'Income')]
data dim: 105
Begin training...
[0/300]	  Loss_D: 0.1798	Loss_G: -0.5250	
[1/300]	  Loss_D: -0.2088	Loss_G: -0.3045	
[2/300]	  Loss_D: 0.0026	Loss_G: -0.2659	
[3/300]	  Loss_D: -0.0133	Loss_G: -0.3154	
[4/300]	  Loss_D: -0.0635	Loss_G: -0.1310	
[5/300]	  Loss_D: -0.1004	Loss_G: -0.2581	
[6/300]	  Loss_D: -0.1050	Loss_G: -0.1945	
[7/300]	  Loss_D: -0.0901	Loss_G: -0.2393	
[8/300]	  Loss_D: 0.0012	Loss_G: -0.2443	
[9/300]	  Loss_D: 0.0152	Loss_G: -0.2704	
[10/300]	  Loss_D: -0.0355	Loss_G: -0.2620	
[11/300]	  Loss_D: -0.0615	Loss_G: -0.2518	
[12/300]	  Loss_D: -0.0561	Loss_G: -0.2670	
[1

In [10]:
y_test =labeled_syn['label'].values
y_score =labeled_syn['y_score'].values

In [11]:
from sklearn.metrics import precision_recall_curve, roc_auc_score
AUCROC = roc_auc_score(y_test, y_score)

In [12]:
# The adversary sets a decision threshold by his choise
T = y_score.mean()+ 2.0*y_score.std()
print(T)

0.6300586789734925


In [13]:
# predict that scores exceed the threshold are sensitive training data
labeled_syn['y_pred'] = y_score > T

In [14]:
# The exposed data of TableGAN-MCA are "Positives", the true postive rate are len(TP)/len(Positives)
Positives = labeled_syn[labeled_syn['y_pred']==1]
TP = Positives[Positives['label']==1]
print(len(TP),len(Positives),len(TP)/len(Positives))

1945 2379 0.8175704077343422
